### Normalization

In [ ]:
from os import listdir
from os.path import isfile, join

data_fn = "/home/jason/Desktop/projects/masters/ml/group_project/Dataset 100/Data/"
onlyfiles = [f for f in listdir(data_fn) if isfile(join(data_fn, f))]

In [ ]:
import torch
from torchvision.datasets.folder import pil_loader
import torchvision.transforms as transforms
from tqdm.notebook import tqdm

use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
transform = transforms.ToTensor()

sum = torch.Tensor([0., 0., 0.]).to(device)
sq_sum = torch.Tensor([0., 0., 0.]).to(device)
count = 0
for fn in tqdm(onlyfiles):
    image = transform(pil_loader(data_fn + fn)).to(device)
    _, n, m = image.shape
    sum += image.sum((1, 2))
    sq_sum += (image**2).sum((1, 2))
    count += n * m

In [ ]:
means = sum/count
stds = torch.sqrt(sq_sum/count - means**2)

In [ ]:
means, stds

### Δεδομένα στο Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Dataset

* transforms
* τροπος που φορτωνονται τα δεδομενα
* augmentation

In [ ]:
import torch.utils.data
import torchvision.transforms as transforms
from torchvision.datasets.folder import pil_loader

class Artists(torch.utils.data.Dataset):
    def __init__(self, base_path, image_ids_fn, images_dir):
        self.base_path = base_path
        self.image_ids_fn = image_ids_fn
        self.images_dir = images_dir
        with open(base_path + image_ids_fn, 'r') as fp:
            rows = list(fp)
            self.fnames = [s.strip().split(',')[0] for s in rows[1:]]
            self.img_class_ids = [int(s.strip().split(',')[1]) for s in rows[1:]]
            self.img_ids = list(range(len(self.fnames)))
        self.transform = transforms.Compose((
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize((0.5138, 0.4915, 0.4315), (0.2675, 0.2572, 0.2626)),
        ))

    
    def __getitem__(self, index):
        img_fname = self.fnames[index]
        image = pil_loader(self.base_path + self.images_dir + img_fname)

        if self.transform is not None:
            image = self.transform(image)
        
        return image, self.img_class_ids[index], self.img_ids[index]

    def __len__(self):
        return len(self.img_ids)


### Μοντέλο

* αρχιτεκτονικη
* βαθος cnn
* βαθος fc
* regularization
* fine-tuning/freeze


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class ResNet34Small(nn.Module):

    def __init__(self, num_classes):
        super(ResNet34Small, self).__init__()
#      ResnetSmall
        # original_model = models.resnet34(pretrained=True)
        # self.features = nn.Sequential(*list(original_model.children())[:-3])
        # self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # self.fc = nn.Linear(256, num_classes)
      # ResnetMedium
        # original_model = models.resnet101(pretrained=True)
        # self.features = nn.Sequential(*list(original_model.children())[:-3])
        # self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # self.fc = nn.Linear(1024, num_classes)
      # ResnetLarge
        original_model = models.resnet152(pretrained=True)
        self.features = nn.Sequential(*list(original_model.children())[:-3])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, num_classes)

      # Inception_v3
        # original_model = models.inception_v3(pretrained=True)
        # self.features = nn.Sequential(*list(original_model.children()))
        # self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # self.fc = nn.Linear(256, num_classes)

      # DenseNet-121
        # original_model = models.densenet121(pretrained=True)
        # self.features = nn.Sequential(*list(original_model.children())[:-3])
        # self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # self.fc = nn.Linear(256, num_classes)


      # DenseNet-161
        # original_model = models.densenet161(pretrained=True)
        # self.features = nn.Sequential(*list(original_model.children())[:-3])
        # self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

class RegNet(nn.Module):

    def __init__(self, num_classes):
        super(RegNet, self).__init__()
        original_model = models.regnet_y_800mf(pretrained=True)
        self.features = nn.Sequential(original_model.stem, *list(original_model.trunk_output.children()))
        self.features.requires_grad = True
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.LazyLinear(256),
            nn.LeakyReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

    def finetune(self, n_layers):
        for child in list((net.features[1]).children())[-n_layers:]:
            child.requires_grad = True

### Training

In [ ]:
from torchvision.transforms.transforms import ToTensor
import os
import argparse

import numpy as np
from sklearn import metrics
from tqdm import tqdm
import torch.nn.functional as F
import torch

def run(net, device, loader, optimizer, split='val', epoch=0, train=False,
        dry_run=False):
    if train:
        net.train()
        torch.set_grad_enabled(True)
    else:
        net.eval()
        torch.set_grad_enabled(False)
    
    loader = tqdm(
        loader,
        ncols=0,
        desc='{1} E{0:02d}'.format(epoch, 'train' if train else 'val')
    )
    
    running_loss = 0
    preds_all = []
    labels_all = []
    for (imgs, img_class_ids) in loader:
        imgs, img_class_ids = (
            imgs.to(device), img_class_ids.to(device).long()
            )
        
        output = net(imgs)
        _, preds = torch.max(output, 1)
        loss = F.cross_entropy(output, img_class_ids)

        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        running_loss += float(loss)
        labels_all.extend(img_class_ids.cpu().numpy())
        preds_all.extend(preds.cpu().numpy())
        
        if dry_run:
            break

    bal_acc = metrics.balanced_accuracy_score(labels_all, preds_all)

    print('Epoch: {}.. '.format(epoch),
        '{} Loss: {:.3f}.. '.format(split, running_loss / len(loader)),
        '{} Accuracy: {:.3f}.. '.format(split, bal_acc),
        )
    
    return running_loss / len(loader)


def train(base_path, net_type, train_ids_fname, val_ids_fname, train_pkl, val_pkl,
          images_dir, model_fname, num_classes=20, batch_size=16, lr=1e-4,
          epochs=20, device='cpu', num_workers=6, dry_run=False):

    with open(base_path + train_pkl,"rb") as f:
      train_dataset = pickle.load(f)
    with open(base_path + val_pkl,"rb") as f:
      val_dataset = pickle.load(f)


    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True,
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=False,
    )

    net = {
        'regnet':RegNet#'resnet':  ResNet34Small
    }[net_type](num_classes)
    net.to(device)
    # Freeze Layers
    k = 0 
    for param in net.features.parameters():
      param.requires_grad = True
    for param in net.features.parameters():
      if(k>25):
        break
      else:
        param.requires_grad = False
        k += 1
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad,
        net.parameters()), lr=lr)
    
    if device == 'cuda':
        torch.backends.cudnn.benchmark = True

    cur_best_val_loss = np.inf
    for epoch in range(epochs):
        _ = run(net, device, train_loader, optimizer, split='train',
                    epoch=epoch, train=True, dry_run=dry_run)
        val_loss = run(net, device, val_loader, optimizer, split='val',
                    epoch=epoch, train=False, dry_run=dry_run)

        if cur_best_val_loss > val_loss:
            if epoch > 0:
                # remove previous best model
                os.remove(model_fname)
            torch.save(net.state_dict(), model_fname)
            cur_best_val_loss = val_loss
        
        if dry_run:
            break

    # load best model for final evaluation
    net = {
        'regnet': RegNet#'resnet': ResNet34Small
    }[net_type](num_classes)
    net.to(device)
    # load best model for final evaluation
    net.load_state_dict(torch.load(model_fname))
    print("\nModel loaded from checkpoint for final evaluation\n")
    run(net, device, val_loader, optimizer, split='val_best', epoch=0,
        train=False)

In [ ]:
import pickle 
#base_path = "/home/jason/Desktop/projects/masters/ml/group_project/Dataset 100/"
base_path = "/content/gdrive/My Drive/art_recognition/Datasets/Dataset 100/"
train_ids_fname = "train_100.csv"
val_ids_fname = "val_100.csv"
train_pkl = "train_100.pkl"
val_pkl = "val_100.pkl"
images_dir = "Data/"
#model_fname = "/home/jason/Desktop/projects/masters/ml/group_project/Dataset 100/resnet.pt"
model_fname = "/content/gdrive/My Drive/art_recognition/Datasets/Dataset 100/regnet-small.pt"
#model_fname = "/content/gdrive/My Drive/art_recognition/Datasets/Dataset 100/resnet-medium.pt"
#model_fname = "/content/gdrive/My Drive/art_recognition/Datasets/Dataset 100/resnet-large.pt"

net_type = "regnet"

use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
net_type = "regnet"
train(base_path, net_type, train_ids_fname, val_ids_fname, train_pkl, val_pkl,
      images_dir, model_fname, device=device, dry_run=False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
train E00: 100% 75/75 [00:19<00:00,  3.88it/s]


Epoch: 0..  train Loss: 2.657..  train Accuracy: 0.264.. 


val E00:   0% 0/25 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
val E00: 100% 25/25 [00:02<00:00,  8.59it/s]


Epoch: 0..  val Loss: 2.141..  val Accuracy: 0.441.. 


train E01:   0% 0/75 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
train E01: 100% 75/75 [00:19<00:00,  3.95it/s]


...

Epoch: 19..  val Loss: 0.956..  val Accuracy: 0.754.. 


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '



Model loaded from checkpoint for final evaluation



val E00:   0% 0/25 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
val E00: 100% 25/25 [00:03<00:00,  8.18it/s]


Epoch: 0..  val_best Loss: 0.826..  val_best Accuracy: 0.777.. 


### Feature extraction από CNN

In [ ]:
import pickle
import numpy
from torchvision.transforms.transforms import ToTensor
import os
import argparse

import numpy as np
from sklearn import metrics
from tqdm import tqdm
import torch.nn.functional as F
import torch
base_path = "/content/gdrive/My Drive/art_recognition/Datasets/Dataset 100/"
train_pkl = "train_100.pkl"
val_pkl = "val_100.pkl"
test_pkl = "test_100.pkl"
net = ResNet34Small(20)
checkpoint = torch.load("/content/gdrive/My Drive/art_recognition/Datasets/Dataset 100/resnet.pt")
net.load_state_dict(checkpoint)
with open(base_path + train_pkl,"rb") as f:
  train_dataset = pickle.load(f)
with open(base_path + test_pkl,"rb") as f:
  test_dataset = pickle.load(f)
with open(base_path + val_pkl,"rb") as f:
  val_dataset = pickle.load(f)

painting_features_train = [(net.avgpool(net.features(x.unsqueeze(0))).flatten().detach().numpy(), y) for x, y in train_dataset]
painting_features_val = [(net.avgpool(net.features(x.unsqueeze(0))).flatten().detach().numpy(), y) for x, y in val_dataset]
painting_features_test = [(net.avgpool(net.features(x.unsqueeze(0))).flatten().detach().numpy(), y) for x, y in test_dataset]
# paintings = {"train":painting_features_train, "test":painting_features_test, "val":painting_features_val} 

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

In [ ]:
output = open('painting_features_train.pkl', 'wb')
pickle.dump(painting_features_train, output)
output.close()
!cp painting_features_train.pkl '/content/gdrive/My Drive/art_recognition/Datasets/Dataset 100/'

In [ ]:
output = open('painting_features_val.pkl', 'wb')
pickle.dump(painting_features_val, output)
output.close()
!cp painting_features_val.pkl '/content/gdrive/My Drive/art_recognition/Datasets/Dataset 100/'

In [ ]:
output = open('painting_features_test.pkl', 'wb')
pickle.dump(painting_features_test, output)
output.close()
!cp painting_features_test.pkl '/content/gdrive/My Drive/art_recognition/Datasets/Dataset 100/'